In [1]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader

# Load pre-trained Camembert model and tokenizer
model_name = "almanach/camembertav2-base"
camembert_model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from datasets import load_dataset, Dataset
dataset = load_dataset("crazyjeannot/fr_literary_dataset_large")

README.md:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

PARAPH_DATASET_FT_400.jsonl:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][0]['query']

'qu’ est -ce que la musique originaire le désir de se jeter à l’ eau se jeta dans la mer de le haut de le rempart après avoir lu toute l’ œuvre de platon qu’ y a -t-il à le fond de le désir de se jeter à l’ eau qu’ y a -t-il à le fond de le désir de s’ immerger dans la chose qui hante? de sauter le pas? de se lancer toutes affaires cessantes à la poursuite déterminée de ce qu’ on ignore de franchir le rubicon de rompre les amarres de s’ affranchir de toutes précautions de se jeter dans la gueule de le loup? de jouer à le fonds étranges expressions qu’ une même ancienneté rassemble toutes ces métaphores de chasse de danse de marine de jeu de guerre sont moins des propositions de la langue naturelle que des figurations de les rêves '

In [ ]:
dataset['train'][0]['pos'][0]

'elles disent toutes l’ imprudence elles disent toutes il n’ a pas cherché à le échapper à le danger qui s’ offrait il est sorti de sa cache il a démissionné de son poste il a quitté son rang '

In [ ]:
from tqdm.notebook import tqdm
from sentence_transformers import InputExample

train_examples = []
first_4000_samples = dataset["train"].select(range(4000))
for sample in tqdm(first_4000_samples):
    query = sample["query"]  # Adjust key names to match your dataset
    positives = sample["pos"]  # List of positive sentences
    negatives = sample["neg"]  # List of negative sentences

    # Create InputExamples for positives
    for positive in positives:
        train_examples.append(InputExample(texts=[query, positive], label=1.0))

    # Create InputExamples for negatives
    for negative in negatives:
        train_examples.append(InputExample(texts=[query, negative], label=0.0))

  0%|          | 0/4000 [00:00<?, ?it/s]

In [ ]:
# Use the Camembert transformer as the base model
word_embedding_model = models.Transformer(model_name)

In [ ]:
# Add a pooling layer for sentence-level embeddings
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,  # Mean pooling
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [ ]:
# Create the SentenceTransformer model
sentence_transformer = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
# Create a DataLoader with a batch size of 16
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [ ]:
# Define the loss
train_loss = losses.MultipleNegativesRankingLoss(model=sentence_transformer)

In [ ]:
# Fine-tune the model
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)  # 10% warmup

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
# Start training
sentence_transformer.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path="./camembert_sentence_transformer"
)

NameError: name 'Dataset' is not defined

In [ ]:
# Save the fine-tuned model
sentence_transformer.save("./camembert_sentence_transformer")
# Test the model
query = "What is Camembert?"
sentences = [
    "Camembert is a type of cheese.",
    "The Eiffel Tower is in Paris.",
    "Cheese is made from milk.",
]

# Encode query and candidate sentences
query_embedding = sentence_transformer.encode(query)
sentence_embeddings = sentence_transformer.encode(sentences)

# Compute cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity([query_embedding], sentence_embeddings)
for sentence, score in zip(sentences, similarities[0]):
    print(f"Sentence: {sentence}, Similarity: {score}")
